<h1>국내 여행 로그 데이터(서부권) (2023)</h1>
<h2>[분석 과정 - Classification(Random Forest)]</h2>
<ul>
<li>데이터 정보 및 전처리 과정은 data_preprocessing.ipynb 참고</li></ul>

<hr>
<h2>[패키지 및 데이터 로드]</h2>

In [18]:
# 데이터를 분석하는데 필요한 패키지 로드하기 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
import re
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False


plt.rcParams.update({'font.family' : 'Malgun Gothic'}) 

# 숫자 표기 설정
pd.options.display.float_format = '{:,.2f}'.format # 지수 없이 숫자로
#pd.reset_option('display.float_format') # 지수표기법

# 출력 설정 변경 (None : 모든 행,열 출력)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
#pd.reset_option('all') # 설정 변경 초기화 

In [19]:
# 병합한 데이터 불러오기
travel_log = pd.read_csv('data/merge.csv', encoding='UTF-8')
travel_log_test = pd.read_csv('data/testmerge_validation.csv', encoding='UTF-8')

[training data]

In [20]:
# 데이터의 행, 열 개수 출력
print('merge.csv의 행, 열 개수 : ')
print(travel_log.shape)

merge.csv의 행, 열 개수 : 
(2341, 35)


In [21]:
# 데이터의 구조와 타입 확인
print('merge.csv의 구조와 타입 : ')
print(travel_log.info())

merge.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341 entries, 0 to 2340
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   GENDER                     2341 non-null   object 
 1   AGE_GRP                    2341 non-null   int64  
 2   JOB_NM                     2341 non-null   int64  
 3   TRAVEL_TERM                2341 non-null   int64  
 4   TRAVEL_LIKE_SGG_1          2341 non-null   int64  
 5   TRAVEL_LIKE_SGG_2          2341 non-null   int64  
 6   TRAVEL_LIKE_SGG_3          2341 non-null   int64  
 7   TRAVEL_STYL_1              2341 non-null   int64  
 8   TRAVEL_STYL_2              2341 non-null   int64  
 9   TRAVEL_STYL_3              2341 non-null   int64  
 10  TRAVEL_STYL_4              2341 non-null   int64  
 11  TRAVEL_STYL_5              2341 non-null   int64  
 12  TRAVEL_STYL_6              2341 non-null   int64  
 13  TRAVEL_STYL_7              

In [22]:
# 데이터의 결측치 확인
print('merge.csv의 결측치 : ')
print(travel_log.isnull().sum())

merge.csv의 결측치 : 
GENDER                       0
AGE_GRP                      0
JOB_NM                       0
TRAVEL_TERM                  0
TRAVEL_LIKE_SGG_1            0
TRAVEL_LIKE_SGG_2            0
TRAVEL_LIKE_SGG_3            0
TRAVEL_STYL_1                0
TRAVEL_STYL_2                0
TRAVEL_STYL_3                0
TRAVEL_STYL_4                0
TRAVEL_STYL_5                0
TRAVEL_STYL_6                0
TRAVEL_STYL_7                0
TRAVEL_STYL_8                0
TRAVEL_STATUS_RESIDENCE      0
TRAVEL_STATUS_DESTINATION    0
TRAVEL_STATUS_ACCOMPANY      0
TRAVEL_MOTIVE                0
TRAVEL_COMPANIONS_NUM        0
TRAVEL_ID                    0
REL_CD                       0
COMPANION_GENDER             0
COMPANION_AGE_GRP            0
TRAVEL_PURPOSE               0
TRAVEL_START_YMD             0
TRAVEL_END_YMD               0
TRAVEL_MISSION_CHECK         0
MAJOR_TRANSPORT              0
TOTAL_MVMN_WON               0
MAJOR_ACTIVITY_TYPE          0
TOTAL_ACTIVITY_WON   

In [23]:
# 데이터 살펴보기
travel_log.head()

,GENDER,AGE_GRP,JOB_NM,TRAVEL_TERM,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE,TRAVEL_COMPANIONS_NUM,TRAVEL_ID,REL_CD,COMPANION_GENDER,COMPANION_AGE_GRP,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,TRAVEL_MISSION_CHECK,MAJOR_TRANSPORT,TOTAL_MVMN_WON,MAJOR_ACTIVITY_TYPE,TOTAL_ACTIVITY_WON,TOTAL_WON,TRAVEL_DURATION,SEASON
0,여,20,12,3,50130,50110,26500,1,1,1,3,3,2,6,6,광주광역시,전남,2인 여행(가족 외),1,1,g_g002178,[8],[1],[3],1;22;4,2023-06-14,2023-06-16,4;1;22,['자가용(승용/승합/트럭 등등)'],"10,000.00",[5],"71,700.00","81,700.00",2,Summer
1,남,40,2,2,48840,48220,26500,1,1,3,6,4,2,7,7,대전광역시,전북,2인 여행(가족 외),1,1,g_g004485,[8],[1],[4],1;3;3;6;7;7;11;12;22;26;23,2023-07-28,2023-07-30,1;3;6,['자가용(승용/승합/트럭 등등)'],"32,600.00",[3],"273,800.00","306,400.00",2,Summer
2,남,20,1,3,46710,45190,47170,2,3,1,4,6,3,1,7,서울특별시,전남,2인 여행(가족 외),7,1,g_g005167,[8],[2],[3],2;21;22;24;3,2023-09-07,2023-09-10,22;24;3,['자가용(승용/승합/트럭 등등)'],"138,400.00",[1],"450,200.00","588,600.00",3,Autumn
3,여,30,3,2,42150,42210,28710,2,2,3,2,3,5,2,6,인천광역시,대전,2인 여행(가족 외),2,1,g_g002887,[1],[2],[4],1;4;9;22;23,2023-07-01,2023-07-03,1;9;23,['자가용(승용/승합/트럭 등등)'],"70,000.00","[1, 2]","119,100.00","189,100.00",2,Summer
4,남,20,4,2,42150,50110,26350,2,1,2,4,6,5,6,5,경기도,충남,2인 여행(가족 외),2,1,g_g004189,[8],[1],[3],22;5;7,2023-07-18,2023-07-20,21;5;22,['자가용(승용/승합/트럭 등등)'],"52,083.00",[1],"120,450.00","172,533.00",2,Summer


[test data]

In [24]:
# 데이터의 행, 열 개수 출력
print('merge_validation.csv의 행, 열 개수 : ')
print(travel_log_test.shape)

merge_validation.csv의 행, 열 개수 : 
(292, 35)


In [25]:
# 데이터의 구조와 타입 확인
print('merge_validation.csv의 구조와 타입 : ')
print(travel_log_test.info())

merge_validation.csv의 구조와 타입 : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   GENDER                     292 non-null    object 
 1   AGE_GRP                    292 non-null    int64  
 2   JOB_NM                     292 non-null    int64  
 3   TRAVEL_TERM                292 non-null    int64  
 4   TRAVEL_LIKE_SGG_1          292 non-null    int64  
 5   TRAVEL_LIKE_SGG_2          292 non-null    int64  
 6   TRAVEL_LIKE_SGG_3          292 non-null    int64  
 7   TRAVEL_STYL_1              292 non-null    int64  
 8   TRAVEL_STYL_2              292 non-null    int64  
 9   TRAVEL_STYL_3              292 non-null    int64  
 10  TRAVEL_STYL_4              292 non-null    int64  
 11  TRAVEL_STYL_5              292 non-null    int64  
 12  TRAVEL_STYL_6              292 non-null    int64  
 13  TRAVEL_STYL_7     

In [26]:
# 데이터의 결측치 확인
print('merge_validation.csv의 결측치 : ')
print(travel_log_test.isnull().sum())

merge_validation.csv의 결측치 : 
GENDER                       0
AGE_GRP                      0
JOB_NM                       0
TRAVEL_TERM                  0
TRAVEL_LIKE_SGG_1            0
TRAVEL_LIKE_SGG_2            0
TRAVEL_LIKE_SGG_3            0
TRAVEL_STYL_1                0
TRAVEL_STYL_2                0
TRAVEL_STYL_3                0
TRAVEL_STYL_4                0
TRAVEL_STYL_5                0
TRAVEL_STYL_6                0
TRAVEL_STYL_7                0
TRAVEL_STYL_8                0
TRAVEL_STATUS_RESIDENCE      0
TRAVEL_STATUS_DESTINATION    0
TRAVEL_STATUS_ACCOMPANY      0
TRAVEL_MOTIVE                0
TRAVEL_COMPANIONS_NUM        0
TRAVEL_ID                    0
REL_CD                       0
COMPANION_GENDER             0
COMPANION_AGE_GRP            0
TRAVEL_PURPOSE               0
TRAVEL_START_YMD             0
TRAVEL_END_YMD               0
TRAVEL_MISSION_CHECK         0
MAJOR_TRANSPORT              0
TOTAL_MVMN_WON               0
MAJOR_ACTIVITY_TYPE          0
TOTAL_ACTI

In [27]:
# 데이터 살펴보기
travel_log_test.head()

,GENDER,AGE_GRP,JOB_NM,TRAVEL_TERM,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MOTIVE,TRAVEL_COMPANIONS_NUM,TRAVEL_ID,REL_CD,COMPANION_GENDER,COMPANION_AGE_GRP,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,TRAVEL_MISSION_CHECK,MAJOR_TRANSPORT,TOTAL_MVMN_WON,MAJOR_ACTIVITY_TYPE,TOTAL_ACTIVITY_WON,TOTAL_WON,TRAVEL_DURATION,SEASON
0,남,30,5,2,42210,44825,41270,1,1,2,1,1,7,7,7,경기도,충남,자녀 동반 여행,2,4,g_g005903,"[1, 3, 2, 11]","[2, 2, 2, 2]","[4, 7, 1, 4]",1;22;26;5,2023-08-12,2023-08-15,5;22;11,['자가용(승용/승합/트럭 등등)'],"119,000.00",[1],"336,010.00","455,010.00",3,Summer
1,여,20,13,2,50130,26350,30140,3,6,6,2,2,5,6,5,충청남도,대전,2인 여행(가족 외),1,1,g_g011387,[7],[2],[3],1;21;22;26;27,2023-09-16,2023-09-17,6;22;1,"['버스 + 지하철', '새마을/무궁화열차']","15,700.00",[2],"102,700.00","118,400.00",1,Autumn
2,여,30,2,2,42810,26710,46130,4,5,3,2,5,3,5,6,인천광역시,대전,3대 동반 여행(친척 포함),7,2,g_g003709,"[3, 11]","[2, 1]","[7, 2]",1;22;24;27,2023-07-05,2023-07-06,22;2;27,['자가용(승용/승합/트럭 등등)'],"76,000.00","[4, 5]","332,800.00","408,800.00",1,Summer
3,남,40,8,2,42210,46710,43800,1,7,1,7,1,1,1,6,경상남도,광주,2인 여행(가족 외),6,1,g_g002674,[8],[2],[4],27;28,2023-06-16,2023-06-18,3;28;27,['자가용(승용/승합/트럭 등등)'],"70,400.00",[4],"66,000.00","136,400.00",2,Summer
4,남,30,3,2,46710,45110,50110,2,6,2,2,1,3,1,2,광주광역시,전남,나홀로 여행,1,0,g_g013104,없음,없음,없음,12;2;21;22;27,2023-09-28,2023-09-29,2;21;22,['자가용(승용/승합/트럭 등등)'],"10,000.00",[3],"18,500.00","28,500.00",1,Autumn


<hr>
<h2>[분석 과정]</h2>

1. 원핫 인코딩 등  전처리 과정

[training data]

In [28]:
# # TRAVEL_ID 중복 여부 확인
# duplicate_ids = travel_log['TRAVEL_ID'].value_counts()

# # 중복된 TRAVEL_ID 필터링 (횟수가 1보다 큰 경우)
# duplicate_ids = duplicate_ids[duplicate_ids > 1]

# # 중복 여부 출력
# if not duplicate_ids.empty:
#     print("중복된 TRAVEL_ID가 있습니다:")
#     print(duplicate_ids)
# else:
#     print("모든 TRAVEL_ID가 고유합니다.")

> TRAVEL_ID가 중복되는 행이 존재하지 않으므로 요약 통계량을 계산할 필요가 없음

In [29]:
# # 선택할 변수 지정
# selected_columns = [
#     'REL_CD', 'COMPANION_GENDER', 'COMPANION_AGE_GRP', 'TRAVEL_ID', 'GENDER', 'AGE_GRP', 'JOB_NM', 'TRAVEL_TERM', 'TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SGG_2', 'TRAVEL_LIKE_SGG_3', 'TRAVEL_STYL_1',
#     'TRAVEL_STYL_2', 'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION', 'TRAVEL_STATUS_ACCOMPANY', 
#     'TRAVEL_MOTIVE', 'TRAVEL_COMPANIONS_NUM', 'TOTAL_MVMN_WON', 'TOTAL_ACTIVITY_WON', 'TOTAL_WON', 'TRAVEL_DURATION', 'SEASON', 'MAJOR_ACTIVITY_TYPE'
# ]

# # 선택된 변수만 포함한 DataFrame 생성
# filtered_data = travel_log[selected_columns]

# # TRAVEL_ID 기준 그룹화
# travel_log_grouped = filtered_data.groupby('TRAVEL_ID').apply(lambda x: x).reset_index(drop=True)

# # 결과 확인
# print(travel_log_grouped.head())

In [30]:
# # MAJOR_ACTIVITY_TYPE: 리스트 형태로 분리 후 원핫 인코딩
# mlb = MultiLabelBinarizer()
# major_activity_type_encoded = pd.DataFrame(mlb.fit_transform(travel_log_grouped['MAJOR_ACTIVITY_TYPE'].apply(eval)),
#                                            columns=mlb.classes_)

# # REL_CD: 리스트 형태로 원핫 인코딩
# mlb_rel_cd = MultiLabelBinarizer()
# rel_cd_encoded = pd.DataFrame(
#     mlb_rel_cd.fit_transform(travel_log_grouped['REL_CD']),
#     columns=[f"REL_CD_{col}" for col in mlb_rel_cd.classes_]
# )

# # COMPANION_GENDER: 리스트 형태로 원핫 인코딩
# mlb_gender = MultiLabelBinarizer()
# companion_gender_encoded = pd.DataFrame(
#     mlb_gender.fit_transform(travel_log_grouped['COMPANION_GENDER']),
#     columns=[f"COMPANION_GENDER_{col}" for col in mlb_gender.classes_]
# )

# # COMPANION_AGE_GRP: 리스트 형태로 원핫 인코딩
# mlb_age_grp = MultiLabelBinarizer()
# companion_age_grp_encoded = pd.DataFrame(
#     mlb_age_grp.fit_transform(travel_log_grouped['COMPANION_AGE_GRP']),
#     columns=[f"COMPANION_AGE_GRP_{col}" for col in mlb_age_grp.classes_]
# )

# # 범주형 변수 원핫 인코딩
# categorical_columns = ['GENDER', 'JOB_NM', 'TRAVEL_TERM', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2',
#                        'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION', 'TRAVEL_STATUS_ACCOMPANY', 
#                        'TRAVEL_MOTIVE', 'SEASON']
# travel_log_encoded = pd.get_dummies(travel_log_grouped[categorical_columns])

# # 수치형 변수 스케일링 (정규화)
# numerical_columns = ['AGE_GRP', 'TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SGG_2', 'TRAVEL_LIKE_SGG_3',
#                      'TRAVEL_COMPANIONS_NUM', 'TOTAL_MVMN_WON', 'TOTAL_ACTIVITY_WON', 'TOTAL_WON', 'TRAVEL_DURATION']
# scaler = StandardScaler()
# travel_log_scaled = pd.DataFrame(scaler.fit_transform(travel_log_grouped[numerical_columns]), columns=numerical_columns)

# # 데이터프레임 합치기
# travel_log_final = pd.concat([travel_log_encoded, travel_log_scaled, major_activity_type_encoded], axis=1)

# # 최종 데이터프레임
# print(travel_log_final.head())


[test data]

In [31]:
# # TRAVEL_ID 중복 여부 확인
# duplicate_ids = travel_log_test['TRAVEL_ID'].value_counts()

# # 중복된 TRAVEL_ID 필터링 (횟수가 1보다 큰 경우)
# duplicate_ids = duplicate_ids[duplicate_ids > 1]

# # 중복 여부 출력
# if not duplicate_ids.empty:
#     print("중복된 TRAVEL_ID가 있습니다:")
#     print(duplicate_ids)
# else:
#     print("모든 TRAVEL_ID가 고유합니다.")

> TRAVEL_ID가 중복되는 행이 존재하지 않으므로 요약 통계량을 계산할 필요가 없음

In [32]:
# # 선택할 변수 지정
# selected_columns = [
#     'REL_CD', 'COMPANION_GENDER', 'COMPANION_AGE_GRP', 'TRAVEL_ID', 'GENDER', 'AGE_GRP', 'JOB_NM', 'TRAVEL_TERM', 'TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SGG_2', 'TRAVEL_LIKE_SGG_3', 'TRAVEL_STYL_1',
#     'TRAVEL_STYL_2', 'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION', 'TRAVEL_STATUS_ACCOMPANY', 
#     'TRAVEL_MOTIVE', 'TRAVEL_COMPANIONS_NUM', 'TOTAL_MVMN_WON', 'TOTAL_ACTIVITY_WON', 'TOTAL_WON', 'TRAVEL_DURATION', 'SEASON', 'MAJOR_ACTIVITY_TYPE'
# ]

# # 선택된 변수만 포함한 DataFrame 생성
# filtered_data = travel_log_test[selected_columns]

# # TRAVEL_ID 기준 그룹화
# travel_log_grouped = filtered_data.groupby('TRAVEL_ID').apply(lambda x: x).reset_index(drop=True)

# # 결과 확인
# print(travel_log_grouped.head())

In [33]:
# # MAJOR_ACTIVITY_TYPE: 리스트 형태로 분리 후 원핫 인코딩
# mlb = MultiLabelBinarizer()
# major_activity_type_encoded = pd.DataFrame(mlb.fit_transform(travel_log_grouped['MAJOR_ACTIVITY_TYPE'].apply(eval)),
#                                            columns=mlb.classes_)

# # REL_CD: 리스트 형태로 원핫 인코딩
# mlb_rel_cd = MultiLabelBinarizer()
# rel_cd_encoded = pd.DataFrame(
#     mlb_rel_cd.fit_transform(travel_log_grouped['REL_CD']),
#     columns=[f"REL_CD_{col}" for col in mlb_rel_cd.classes_]
# )

# # COMPANION_GENDER: 리스트 형태로 원핫 인코딩
# mlb_gender = MultiLabelBinarizer()
# companion_gender_encoded = pd.DataFrame(
#     mlb_gender.fit_transform(travel_log_grouped['COMPANION_GENDER']),
#     columns=[f"COMPANION_GENDER_{col}" for col in mlb_gender.classes_]
# )

# # COMPANION_AGE_GRP: 리스트 형태로 원핫 인코딩
# mlb_age_grp = MultiLabelBinarizer()
# companion_age_grp_encoded = pd.DataFrame(
#     mlb_age_grp.fit_transform(travel_log_grouped['COMPANION_AGE_GRP']),
#     columns=[f"COMPANION_AGE_GRP_{col}" for col in mlb_age_grp.classes_]
# )

# # 범주형 변수 원핫 인코딩
# categorical_columns = ['GENDER', 'JOB_NM', 'TRAVEL_TERM', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2',
#                        'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION', 'TRAVEL_STATUS_ACCOMPANY', 
#                        'TRAVEL_MOTIVE', 'SEASON']
# travel_log_encoded = pd.get_dummies(travel_log_grouped[categorical_columns])

# # 수치형 변수 스케일링 (정규화)
# numerical_columns = ['AGE_GRP', 'TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SGG_2', 'TRAVEL_LIKE_SGG_3',
#                      'TRAVEL_COMPANIONS_NUM', 'TOTAL_MVMN_WON', 'TOTAL_ACTIVITY_WON', 'TOTAL_WON', 'TRAVEL_DURATION']
# scaler = StandardScaler()
# travel_log_scaled = pd.DataFrame(scaler.fit_transform(travel_log_grouped[numerical_columns]), columns=numerical_columns)

# # 데이터프레임 합치기
# travel_log_final_test = pd.concat([travel_log_encoded, travel_log_scaled, major_activity_type_encoded], axis=1)

# # 최종 데이터프레임
# print(travel_log_final_test.head())


2. Random Forest

In [34]:
# import pandas as pd
# from sklearn.preprocessing import OneHotEncoder
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_squared_error, r2_score

# # Feature와 Target 분리
# X_train = travel_log.drop(['TOTAL_WON'], axis=1)
# y_train = travel_log['TOTAL_WON']

# X_test = travel_log_test.drop(['TOTAL_WON'], axis=1)
# y_test = travel_log_test['TOTAL_WON']

# # 1. 데이터 타입 일치
# X_test = X_test.astype(X_train.dtypes)

# # 2. 열 이름 정리: 공백 제거
# X_train.columns = X_train.columns.str.strip()
# X_test.columns = X_test.columns.str.strip()

# # 3. 원핫 인코딩 일관성 점검
# encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# # 훈련 데이터와 테스트 데이터를 합쳐 원핫 인코딩
# combined_data = pd.concat([X_train, X_test], axis=0)
# encoded_combined_data = encoder.fit_transform(combined_data)

# # 원핫 인코딩된 열 이름 가져오기
# encoded_columns = encoder.get_feature_names_out(X_train.columns)

# # 훈련 데이터와 테스트 데이터로 다시 분리
# X_train_encoded = pd.DataFrame(encoded_combined_data[:len(X_train)], columns=encoded_columns)
# X_test_encoded = pd.DataFrame(encoded_combined_data[len(X_train):], columns=encoded_columns)

# # 인코딩 후 결측값 처리
# X_train_encoded = X_train_encoded.fillna(0)
# X_test_encoded = X_test_encoded.fillna(0)


In [35]:
# # XGBRegressor 모델 생성 및 학습
# xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
# xgb_model.fit(X_train_encoded, y_train)

# # 예측
# y_pred_train = xgb_model.predict(X_train_encoded)
# y_pred_test = xgb_model.predict(X_test_encoded)

# # 모델 평가
# mse_train = mean_squared_error(y_train, y_pred_train)
# r2_train = r2_score(y_train, y_pred_train)

# mse_test = mean_squared_error(y_test, y_pred_test)
# r2_test = r2_score(y_test, y_pred_test)

# print("Training Mean Squared Error (MSE):", mse_train)
# print("Training R² Score:", r2_train)
# print("Test Mean Squared Error (MSE):", mse_test)
# print("Test R² Score:", r2_test)


In [36]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Feature와 Target 분리
X_train = travel_log.drop(['TOTAL_WON'], axis=1)  # Feature 데이터
y_train = travel_log['TOTAL_WON']  # Target 데이터

X_test = travel_log_test.drop(['TOTAL_WON'], axis=1)  # Test Feature 데이터
y_test = travel_log_test['TOTAL_WON']  # Test Target 데이터

# XGBoost Regressor 모델 생성
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# 예측
y_pred_train = xgb_model.predict(X_train)
y_pred_test = xgb_model.predict(X_test)

# 모델 평가
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("Training Mean Squared Error (MSE):", mse_train)
print("Training R² Score:", r2_train)
print("Test Mean Squared Error (MSE):", mse_test)
print("Test R² Score:", r2_test)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:GENDER: object, TRAVEL_STATUS_RESIDENCE: object, TRAVEL_STATUS_DESTINATION: object, TRAVEL_STATUS_ACCOMPANY: object, TRAVEL_ID: object, REL_CD: object, COMPANION_GENDER: object, COMPANION_AGE_GRP: object, TRAVEL_PURPOSE: object, TRAVEL_START_YMD: object, TRAVEL_END_YMD: object, TRAVEL_MISSION_CHECK: object, MAJOR_TRANSPORT: object, MAJOR_ACTIVITY_TYPE: object, SEASON: object

In [ ]:
# 훈련 데이터에 예측값 추가
travel_log_final['Predicted_Spending'] = y_pred_train

# 고소비/중소비/저소비 그룹화 함수
def classify_spending(amount):
    if amount > travel_log_final['Predicted_Spending'].quantile(0.75):  # 상위 25%
        return 'High Spender'
    elif amount < travel_log_final['Predicted_Spending'].quantile(0.25):  # 하위 25%
        return 'Low Spender'
    else:
        return 'Medium Spender'

travel_log_final['Spender_Type'] = travel_log_final['Predicted_Spending'].apply(classify_spending)

# 그룹별 통계 확인
spender_summary = travel_log_final.groupby('Spender_Type')['Predicted_Spending'].mean()
print(spender_summary)

# 시각화
import matplotlib.pyplot as plt

spender_summary.plot(kind='bar', color=['gold', 'lightblue', 'pink'])
plt.title('Average Predicted Spending by Spender Type')
plt.xlabel('Spender Type')
plt.ylabel('Average Spending')
plt.show()


In [ ]:
# 지역별 평균 예측 소비 금액 계산
regional_spending = travel_log_final.groupby('TRAVEL_STATUS_DESTINATION')['Predicted_Spending'].mean()

# 결과 시각화
plt.figure(figsize=(12, 6))
regional_spending.sort_values(ascending=False).plot(kind='bar', color='skyblue')
plt.title('Average Predicted Spending by Destination')
plt.xlabel('Travel Destination')
plt.ylabel('Average Predicted Spending')
plt.xticks(rotation=45)
plt.show()


In [ ]:
from sklearn.cluster import KMeans

# K-Means 군집화
kmeans = KMeans(n_clusters=3, random_state=42)
travel_log_final['Cluster'] = kmeans.fit_predict(X_train)

# 군집별 평균 소비 금액
cluster_spending = travel_log_final.groupby('Cluster')['Predicted_Spending'].mean()

# 시각화
plt.figure(figsize=(10, 5))
cluster_spending.plot(kind='bar', color='orange')
plt.title('Average Predicted Spending by Travel Style (Cluster)')
plt.xlabel('Cluster')
plt.ylabel('Average Predicted Spending')
plt.xticks(rotation=45)
plt.show()


In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

# PDF 파일 생성
with PdfPages('Travel_Spending_Report.pdf') as pdf:
    # 그래프 1: 지역별 소비 금액
    plt.figure(figsize=(12, 6))
    regional_spending.sort_values(ascending=False).plot(kind='bar', color='skyblue')
    plt.title('Average Predicted Spending by Destination')
    plt.xlabel('Travel Destination')
    plt.ylabel('Average Predicted Spending')
    plt.xticks(rotation=45)
    pdf.savefig()  # 그래프 저장
    plt.close()

    # 그래프 2: 시즌별 소비 금액
    plt.figure(figsize=(10, 5))
    seasonal_spending.plot(kind='bar', color='lightgreen')
    plt.title('Average Predicted Spending by Season')
    plt.xlabel('Season')
    plt.ylabel('Average Predicted Spending')
    plt.xticks(rotation=45)
    pdf.savefig()  # 그래프 저장
    plt.close()

    # 그래프 3: 군집별 소비 금액
    plt.figure(figsize=(10, 5))
    cluster_spending.plot(kind='bar', color='orange')
    plt.title('Average Predicted Spending by Travel Style (Cluster)')
    plt.xlabel('Cluster')
    plt.ylabel('Average Predicted Spending')
    plt.xticks(rotation=45)
    pdf.savefig()  # 그래프 저장
    plt.close()


In [47]:
# 열 이름 확인
print("Columns in travel_log_final:", travel_log_final.columns)

# KeyError 방지: 존재 여부 확인
if 'TRAVEL_STATUS_DESTINATION' in travel_log_final.columns:
    X_train = travel_log_final.drop(['TRAVEL_STATUS_DESTINATION'], axis=1)
    y_train = travel_log_final['TRAVEL_STATUS_DESTINATION']
else:
    print("Error: 'TRAVEL_STATUS_DESTINATION' 열이 데이터프레임에 없습니다.")

Columns in travel_log_final: Index(['GENDER_남', 'GENDER_여', 'JOB_NM_관리자', 'JOB_NM_군인', 'JOB_NM_기능원 및 관련 기능 종사자', 'JOB_NM_기타', 'JOB_NM_농림어업 숙련 종사자', 'JOB_NM_단순노무종사자', 'JOB_NM_사무 종사자', 'JOB_NM_서비스 종사자',
       ...
       'TOTAL_WON', 'TRAVEL_DURATION', '기타 활동', '단순 구경 / 산책 / 걷기', '쇼핑 / 구매', '없음', '체험 활동 / 입장 및 관람', '취식', '환승/경유', '휴식'], dtype='object', length=103)
Error: 'TRAVEL_STATUS_DESTINATION' 열이 데이터프레임에 없습니다.


In [50]:
from xgboost import XGBRegressor

# Feature와 Target
X_train = travel_log_final.drop(['TOTAL_WON'], axis=1)
y_train = travel_log_final['TOTAL_WON']

# XGBoost 모델
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# 예측 및 평가
y_pred_train = xgb_model.predict(X_train)
print("Training Predictions:", y_pred_train[:5])


Training Predictions: [ 1.8808719  -0.35473803  0.4372527   1.9587966   0.4735594 ]


In [ ]:
# Feature와 Target 분리
X_train = travel_log_final.drop(['TRAVEL_STATUS_DESTINATION'], axis=1)  # Feature 데이터
y_train = travel_log_final['TRAVEL_STATUS_DESTINATION']  # Target 데이터

X_test = travel_log_final_test.drop(['TRAVEL_STATUS_DESTINATION'], axis=1)  # Test Feature 데이터
y_test = travel_log_final_test['TRAVEL_STATUS_DESTINATION']  # Test Target 데이터

# Random Forest 모델 생성 및 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 예측
y_pred_train = rf_model.predict(X_train)
y_pred_test = rf_model.predict(X_test)

# 결과 평가
print("Random Forest - Training Accuracy:", accuracy_score(y_train, y_pred_train))
print("Random Forest - Testing Accuracy:", accuracy_score(y_test, y_pred_test))
print("\nClassification Report (Test Data):\n", classification_report(y_test, y_pred_test))

# 중요 변수 확인
feature_importances = pd.DataFrame({'Feature': X_train.columns, 'Importance': rf_model.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print("\nFeature Importances:\n", feature_importances.head(10))

KeyError: "['TRAVEL_STATUS_DESTINATION'] not found in axis"